In [43]:
import pandas as pd
import numpy as np 
import os , sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from database.db_config import Connection

In [44]:
query = """

WITH cte As (
SELECT 
    stock_symbol, timestamp, close,
    AVG(close) OVER(PARTITION BY stock_symbol ORDER BY timestamp ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS ma_5,
    AVG(close) OVER (PARTITION BY stock_symbol ORDER BY  timestamp ROWS BETWEEN 9 PRECEDING AND CURRENT ROW) AS  ma_10,
    AVG(close)  OVER (PARTITION BY  stock_symbol ORDER BY  timestamp ROWS BETWEEN 49 PRECEDING AND CURRENT ROW ) AS ma_50

FROM stock_prices_daily
)

UPDATE  stock_prices_daily spd

SET ma_5= cte.ma_5, ma_10= cte.ma_10, ma_50= cte.ma_50

From cte

WHERE spd.stock_symbol = cte.stock_symbol  AND spd.timestamp = cte.timestamp;

"""

db= Connection()
db.connect()


db.execute(query=query)







✅ Connected to the database


In [45]:
return_query= """
WITH prev_day AS (
SELECT 
    stock_symbol,timestamp, close,
    LAG(close) OVER( PARTITION BY stock_symbol ORDER BY timestamp) AS prev_close

FROM stock_prices_daily 

)

UPDATE stock_prices_daily  spd

SET daily_return = (prev_day.close - prev_day.prev_close)/ prev_day.prev_close

FROM prev_day 

WHERE spd.stock_symbol = prev_day.stock_symbol AND  spd.timestamp = prev_day.timestamp;

"""

db= Connection()
db.connect()


ds=db.execute(return_query)


print(ds)





✅ Connected to the database
None


In [38]:
test_query_2 = """   
WITH intraday_aggregated AS (
    SELECT 
        stock_symbol,
        DATE(timestamp) AS date,  -- Extracts only the date (removing time)
        AVG(close) AS avg_closing_intraday,
        STDDEV(close) AS intraday_volatility
    FROM stock_prices_intraday
   -- WHERE DATE(timestamp) = '2025-01-31'  -- ✅ Only process existing intraday data
    GROUP BY stock_symbol, DATE(timestamp)
    
)

UPDATE stock_prices_daily spd
SET 
    avg_closing_intraday = ia.avg_closing_intraday,
    intraday_volatility = ia.intraday_volatility
FROM intraday_aggregated ia
WHERE spd.stock_symbol = ia.stock_symbol 
AND spd.timestamp::date = ia.date
RETURNING spd.*;




"""

In [33]:
join_quer= """   
SELECT spd.timestamp::date, ia.date, ia.avg_closing_intraday, ia.intraday_volatility
FROM stock_prices_daily spd
JOIN (
    SELECT stock_symbol, DATE(timestamp) AS date, AVG(close) AS avg_closing_intraday, STDDEV(close) AS intraday_volatility
    FROM stock_prices_intraday
    
    GROUP BY stock_symbol, DATE(timestamp)
) ia 
ON spd.stock_symbol = ia.stock_symbol 
AND spd.timestamp::date = ia.date;



"""

In [41]:
intrday_query= """ 
WITH intraday_quer AS (
    SELECT 
        stock_symbol,
        DATE(timestamp) AS date,
        AVG(close) AS avg_closing_intraday,
        STDDEV(close) AS intraday_volatility

    FROM  stock_prices_intraday
    GROUP BY  stock_symbol ,DATE(timestamp)
)

UPDATE stock_prices_daily spd

SET 
    avg_closing_intraday= intraday_quer.avg_closing_intraday, 
    intraday_volatility= intraday_quer.intraday_volatility

FROM intraday_quer

WHERE 
    spd.stock_symbol = intraday_quer.stock_symbol 
    AND spd.timestamp::date= intraday_quer.date;

"""

db.connect()
db.execute(intrday_query)

✅ Connected to the database


In [46]:
quer= "select intraday_volatility  from stock_prices_daily WHERE timestamp = '2025-01-31 00:00:00+05';"

db= Connection()
db.connect()

dtt=db.fetch(quer)


df=pd.DataFrame(dtt)



print(df)






✅ Connected to the database
          0
0  0.420688


In [ ]:
quer= """   
WITH cte AS (
    SELECT 
        stock_symbol,
        DATE(timestamp) as date,
        AVG(close) as average_closing_intraday
    FROM stock_prices_intraday
    GROUP BY stock_symbol,DATE(timestamp)
)

UPDATE stock_prices_daily spd

SET avg_closing_intraday= ia.average_closing_intraday

FROM cte ia

WHERE spd.stoc_symbol= ia.stock_symbol



"""

In [42]:
te_quer="""-- Check latest stock data with moving averages
SELECT * FROM stock_prices_daily ORDER BY timestamp DESC LIMIT 10;

-- Verify intraday volatility and average closing values
SELECT stock_symbol, timestamp, avg_closing_intraday, intraday_volatility 
FROM stock_prices_daily 
ORDER BY timestamp DESC LIMIT 10;"""

db=Connection()

db.connect()
dt=db.fetch(te_quer)

df=pd.DataFrame(dt)

print(df )




✅ Connected to the database
     0                         1           2         3
0  IBM 2025-01-31 00:00:00+05:00  255.541487  0.420688
1  IBM 2025-01-30 00:00:00+05:00         NaN       NaN
2  IBM 2025-01-29 00:00:00+05:00         NaN       NaN
3  IBM 2025-01-28 00:00:00+05:00         NaN       NaN
4  IBM 2025-01-27 00:00:00+05:00         NaN       NaN
5  IBM 2025-01-24 00:00:00+05:00         NaN       NaN
6  IBM 2025-01-23 00:00:00+05:00         NaN       NaN
7  IBM 2025-01-22 00:00:00+05:00         NaN       NaN
8  IBM 2025-01-21 00:00:00+05:00         NaN       NaN
9  IBM 2025-01-17 00:00:00+05:00         NaN       NaN


In [ ]:
import sys
import os 

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from database.db_config import Connection

db= Connection()

db.connect()

query= """
WITH prev_day AS(
    SELECT 
        stock_symbol,
        timestamp,
        volume,
        close,
        open
    

    FROM stock_prices_daily 
)

    SELECT
        d.stock_symbol,
        d.timestamp,
        d.close,
        d.open,
        d.high,
        d.low
    
    FROM stock_prices_daily d
    
    LEFT JOIN prev_day

    ON d.stock_symbol= prev_day.stock_symbol AND d.timestamp = prev_day.timestamp + INTERVAL '1 day'
    ORDER BY d.stock_symbol , d.timestamp
"""

data=db.fetch(query=query)
import pandas as pd

df=pd.DataFrame(data=data)

print(df)



✅ Connected to the database
        0                          1       2       3        4       5
0     IBM  1999-11-01 00:00:00+05:00   96.75   98.50   98.810   96.37
1     IBM  1999-11-02 00:00:00+05:00   94.81   96.75   96.810   93.69
2     IBM  1999-11-03 00:00:00+05:00   94.37   95.87   95.940   93.50
3     IBM  1999-11-04 00:00:00+05:00   91.56   94.44   94.440   90.00
4     IBM  1999-11-05 00:00:00+05:00   90.25   92.75   92.940   90.19
...   ...                        ...     ...     ...      ...     ...
6347  IBM  2025-01-27 00:00:00+05:00  224.13  222.19  224.300  219.84
6348  IBM  2025-01-28 00:00:00+05:00  225.66  224.32  225.770  221.77
6349  IBM  2025-01-29 00:00:00+05:00  228.63  225.62  229.470  223.73
6350  IBM  2025-01-30 00:00:00+05:00  258.27  250.00  261.800  247.01
6351  IBM  2025-01-31 00:00:00+05:00  255.70  256.05  257.235  251.89

[6352 rows x 6 columns]
